In [3]:
import os
import numpy as np
from scipy.stats import mstats
import mat73


In [4]:
f = os.path.join('VPdx.mat')
data = mat73.loadmat(f)


In [17]:
cntSize = data['cnt']['x'].shape[0]
numEvents = data['mrk']['time'].shape[0]
ad_size = cntSize
altered_data = np.zeros((ad_size, 65))

srd = 5
adjustedTime = np.array([x/srd for x in data['mrk']['time']])

print(adjustedTime.shape)

(1670,)


In [29]:
numEEG = 62
ppSub = np.mean(data['cnt']['x'][:100, :numEEG], axis=0)

print(ppSub.shape)
for pp in range(100, cntSize):
    data['cnt']['x'][pp:pp+1, :numEEG] = data['cnt']['x'][pp:pp+1, :numEEG] - ppSub

print(data['cnt']['x'][0])

(62,)
[ 3.03167173e+08 -6.00104435e+08 -8.55798319e+08 -8.47012855e+08
 -8.65192405e+08 -1.65323922e+08 -5.68740007e+08 -5.63770174e+08
 -8.56924500e+08 -5.90753320e+08 -8.48420971e+08 -8.54688422e+08
 -6.12356056e+08 -8.70529025e+08 -6.33708663e+08 -8.35923638e+08
 -5.69677492e+08 -8.69454311e+08 -8.62736605e+08 -5.78923601e+08
 -8.64928341e+08 -5.99791480e+08 -6.34318135e+08 -8.52428585e+08
 -5.78729174e+08 -8.71453343e+08 -8.72853827e+08 -5.68844035e+08
 -8.30382456e+08 -5.87802791e+08 -6.17268310e+08 -8.65367939e+08
 -5.42230390e+08 -8.70856343e+08 -8.57142860e+08 -5.77571808e+08
 -8.46872787e+08 -5.84287970e+08 -5.72617517e+08 -8.56286036e+08
 -5.96534056e+08 -8.73001748e+08 -8.65894624e+08 -5.93530923e+08
 -8.71556679e+08 -5.83422288e+08 -5.84197744e+08 -8.52733546e+08
 -5.70111876e+08 -8.45737344e+08 -8.71361322e+08 -5.90914884e+08
 -8.93456106e+08 -6.24564438e+08 -6.09974697e+08 -8.69625705e+08
 -6.12072380e+08 -8.84066155e+08 -8.92124770e+08 -6.07120873e+08
 -8.93045603e+08 -2

In [28]:
itr = 1
for fill in range(0, cntSize):
        altered_data[itr:itr+1,1:numEEG] = data['cnt']['x'][fill:fill+1,1:numEEG] # the eeg data
        altered_data[itr:itr+1,numEEG+1] = srd*itr
        itr = itr + 1


In [32]:
for j in range(0, numEvents):
    if data['mrk']['y'][1][j] == 1: # 5 indicates we're finding the target based off react_emg, 2 indicates we're finding the target based on the lead care braking
        approx_index = adjustedTime[j]
        for k in range(int(approx_index-round((1300/srd))), int(approx_index+round(200/srd))): # The target duration at the time of react_emg is -1300 ms and +200 ms or -300 ms and + 1200
            altered_data[k][numEEG+2] = 1; #label as braking
        

In [34]:
print(altered_data.shape)

(1619949, 65)


In [37]:
# extract features for training and testing
targetSize = int((1500/srd)) # 1500 miliseconds, divided by srd, divided by downsampling amount
numCategories = 5
numFR = numCategories*numEEG+1
features = np.zeros((int(ad_size/targetSize), numFR))

In [51]:
itr = 1;
m = 1;
while m < ad_size-3*targetSize:
    spaceCheck = m+int((3000/srd)) # 3000 miliseconds ahead to make sure there are no braking events ahead

    if (altered_data[spaceCheck,numEEG+2] == 1): # there is a braking event coming up
        m = m+round((3000/srd)) # skip to the known braking point
        #label
        features[itr,numFR] = 1;

        # features
        ADwindow = altered_data[m:m+targetSize,1:numEEG]
        # mean
        features[itr,0:numEEG] = np.mean(ADwindow);      
        # standard deviation
        features[itr,numEEG+1:2*numEEG] = np.std(ADwindow)
        # activity
        features[itr,2*numEEG+1:3*numEEG] = np.var(ADwindow)
        # mobility and complexity
        for d in range(1, numEEG):
            ADwd0 = ADwindow[:d]
            ADwd1 = np.zeros(1,len(ADwd0))
            ADwd2 = np.zeros(1,len(ADwd0))
            for d1 in range(2,len(ADwd0)):
                ADwd1[d1] = ADwd0[d1] - ADwd0[d1-1]
            
            # mobility
            features[itr,3*numEEG+d] = np.std(ADwd1)/np.std(ADwd0); 
            for d2 in (2, len(ADwd0)):
                ADwd2[d2] = ADwd0(d2) - 2*ADwd0(d2-1) + ADwd0(d2-2)
            
            FF = (np.std(ADwd2)/np.std(ADwd1))/(np.std(ADwd1)/np.std(ADwd0))
            # complexity
            features[itr,4*numEEG+d] = FF;
                      
        m = m+1 + targetSize + int((3000/srd)) # move end not only to then end of the target but also 3000ms ahead of it

    else: # this is a non braking event
        # label
        features[itr, numFR] = 0

        # features 
        ADwindow = altered_data[m:m+targetSize,1:numEEG]
        # mean
        features[itr,1:numEEG] = np.mean(ADwindow);       
        # standard deviation
        features[itr,numEEG+1:2*numEEG] = np.std(ADwindow)
        # activity
        features[itr,2*numEEG+1:3*numEEG] = np.var(ADwindow)
        # mobility and complexity
        for d in range(0, numEEG):
            ADwd0 = ADwindow[:d]
            ADwd1 = np.zeros(1,len(ADwd0))
            ADwd2 = np.zeros(1,len(ADwd0))
            for d1 in range(1, len(ADwd0)):
                ADwd1[d1] = ADwd0[d1] - ADwd0[d1-1]
            
            # mobility
            features[itr,3*numEEG+d] = np.std(ADwd1)/np.std(ADwd0);
            for d2 in range(2,len(ADwd0)):
                ADwd2[d2] = ADwd0[d2] - 2*ADwd0[d2-1] + ADwd0[d2-2]
            
            FF = (np.std(ADwd2)/np.std(ADwd1))/(np.std(ADwd1)/np.std(ADwd0))
            # complexity
            features[itr,4*numEEG+d] = FF
                      
        m = m+1 + targetSize
    itr = itr + 1


IndexError: index 311 is out of bounds for axis 1 with size 311